# Query Data with AWS Data Wrangler

**AWS Data Wrangler** is an open-source Python library that extends the power of the Pandas library to AWS connecting DataFrames and AWS data related services (Amazon Redshift, AWS Glue, Amazon Athena, Amazon EMR, Amazon QuickSight, etc).

* https://github.com/awslabs/aws-data-wrangler
* https://aws-data-wrangler.readthedocs.io

Built on top of other open-source projects like Pandas, Apache Arrow, Boto3, s3fs, SQLAlchemy, Psycopg2 and PyMySQL, it offers abstracted functions to execute usual ETL tasks like load/unload data from Data Lakes, Data Warehouses and Databases.

_Note that AWS Data Wrangler is simply a Python library that uses existing AWS Services.  AWS Data Wrangler is not a separate AWS Service.  You install AWS Data Wrangler through `pip install` as we will see next._

# _Pre-Requisite: Make Sure You Created an Athena Table for Both TSV and Parquet in Previous Notebooks_

In [1]:
%store -r ingest_create_athena_table_tsv_passed

In [2]:
try:
    ingest_create_athena_table_tsv_passed
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.')
    print('++++++++++++++++++++++++++++++++++++++++++++++')

In [3]:
print(ingest_create_athena_table_tsv_passed)

True


In [4]:
if not ingest_create_athena_table_tsv_passed:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.')
    print('++++++++++++++++++++++++++++++++++++++++++++++')
else:
    print('[OK]')

[OK]


In [5]:
%store -r ingest_create_athena_table_parquet_passed

In [6]:
try:
    ingest_create_athena_table_parquet_passed
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not convert into Parquet data.')
    print('++++++++++++++++++++++++++++++++++++++++++++++')

In [7]:
print(ingest_create_athena_table_parquet_passed)

True


In [8]:
if not ingest_create_athena_table_parquet_passed:
    print('++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not convert into Parquet data.') 
    print('++++++++++++++++++++++++++++++++++++++++++++++')
else:
    print('[OK]')

[OK]


# Setup

In [9]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

In [10]:
import awswrangler as wr

# Query Parquet from S3 with Push-Down Filters

Read Apache Parquet file(s) from from a received S3 prefix or list of S3 objects paths.

The concept of Dataset goes beyond the simple idea of files and enable more complex features like partitioning and catalog integration (AWS Glue Catalog): 

_dataset (bool)_ – If True read a parquet dataset instead of simple file(s) loading all the related partitions as columns.

In [11]:
p_filter = lambda x: True if x["product_category"] == "Digital_Software" else False

In [12]:
path = 's3://{}/amazon-reviews-pds/parquet/'.format(bucket)
df_parquet_results = wr.s3.read_parquet(path,
                                        columns=['star_rating', 'product_category', 'review_body'],
                                        partition_filter=p_filter,
                                        dataset=True)
df_parquet_results.shape

(102084, 3)

In [13]:
df_parquet_results.head(5)

,star_rating,review_body,product_category
0,2,I chose the deluxe version CD because of mortg...,Digital_Software
1,4,"Schedule C IS for business, so figures it wou...",Digital_Software
2,2,I wish that companies can test several scenari...,Digital_Software
3,3,i just installed turbotax deluxe 2007. If you ...,Digital_Software
4,1,The description mentions that you can use this...,Digital_Software


# Query Parquet from S3 in Chunks

Batching (chunked argument) (Memory Friendly):

Will enable the function to return a Iterable of DataFrames instead of a regular DataFrame.

There are two batching strategies on Wrangler:
* If chunked=True, a new DataFrame will be returned for each file in your path/dataset.
* If chunked=INTEGER, Wrangler will iterate on the data by number of rows equal to the received INTEGER.

P.S. chunked=True if faster and uses less memory while chunked=INTEGER is more precise in number of rows for each Dataframe.

In [14]:
path = 's3://{}/amazon-reviews-pds/parquet/'.format(bucket)
chunk_iter = wr.s3.read_parquet(path,
                                columns=['star_rating', 'product_category', 'review_body'],
                                # filters=[("product_category", "=", "Digital_Software")],
                                partition_filter=p_filter,
                                dataset=True,
                                chunked=True)

In [15]:
print(next(chunk_iter))

   star_rating                                        review_body  \
0            2  I chose the deluxe version CD because of mortg...   
1            4  Schedule C  IS for business, so figures it wou...   
2            2  I wish that companies can test several scenari...   
3            3  i just installed turbotax deluxe 2007. If you ...   
4            1  The description mentions that you can use this...   

   product_category  
0  Digital_Software  
1  Digital_Software  
2  Digital_Software  
3  Digital_Software  
4  Digital_Software  


# Query the Glue Catalog (ie. Hive Metastore)
Get an iterator of tables.

In [16]:
database_name = 'dsoaws'
table_name_tsv = 'amazon_reviews_tsv'
table_name_parquet = 'amazon_reviews_parquet'

In [17]:
for table in wr.catalog.get_tables(database="dsoaws"):
    print(table['Name'])

amazon_reviews_parquet
amazon_reviews_tsv


# Query from Athena
Execute any SQL query on AWS Athena and return the results as a Pandas DataFrame.  


In [18]:
%%time
df = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_parquet),
    database=database_name
)

CPU times: user 456 ms, sys: 11.9 ms, total: 468 ms
Wall time: 8.37 s


In [19]:
df.head(5)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,year,review_date,product_category
0,US,41754720,R19OFJV91M7D8X,B000YMR61A,141393130,TurboTax Deluxe Federal + State 2007,2,12,13,N,N,"Easy to use, 1 comment 1 serious problem",I chose the deluxe version CD because of mortg...,2008,2008-02-11,Digital_Software
1,US,51669529,R1I6G894K5AGG5,B000YMR61A,141393130,TurboTax Deluxe Federal + State 2007,4,6,9,N,N,Schedule C IS for business- figures it would ...,"Schedule C IS for business, so figures it wou...",2008,2008-02-08,Digital_Software
2,US,24731012,R17OE43FFEP81I,B000YMR5X4,234295632,TurboTax Premier Federal + State 2007,2,9,16,N,N,Hassel to download,I wish that companies can test several scenari...,2008,2008-02-05,Digital_Software
3,US,16049580,R15MGDDK63B52Z,B000YMR61A,141393130,TurboTax Deluxe Federal + State 2007,3,14,14,N,N,beware of vista,i just installed turbotax deluxe 2007. If you ...,2008,2008-02-05,Digital_Software
4,US,46098046,R1GGJJA2R68033,B000YMNI2Q,847631772,TurboTax Basic 2007,1,54,60,N,N,don't waste your money,The description mentions that you can use this...,2008,2008-01-26,Digital_Software


# Query from Athena in Chunks
Retrieving in chunks can help reduce memory requirements.  

_This will take a few seconds._

In [20]:
%%time

chunk_iter = wr.athena.read_sql_query(
    sql='SELECT * FROM {} LIMIT 5000'.format(table_name_parquet),
    database='{}'.format(database_name),
    chunksize=64_000  # 64 KB Chunks
)

CPU times: user 193 ms, sys: 10.9 ms, total: 204 ms
Wall time: 6.19 s


In [21]:
print(next(chunk_iter))

     marketplace customer_id       review_id  product_id product_parent  \
0             US    41754720  R19OFJV91M7D8X  B000YMR61A      141393130   
1             US    51669529  R1I6G894K5AGG5  B000YMR61A      141393130   
2             US    24731012  R17OE43FFEP81I  B000YMR5X4      234295632   
3             US    16049580  R15MGDDK63B52Z  B000YMR61A      141393130   
4             US    46098046  R1GGJJA2R68033  B000YMNI2Q      847631772   
...          ...         ...             ...         ...            ...   
4995          US    49184630  R2Z5HD93KKY3UF  B006WD9H9Y      503749355   
4996          US    17204558  R3J638UEX1MB5T  B004VNHWEO      470252350   
4997          US    51993618   RCFEXJ9O9AGS1  B0052YFO2S      572070203   
4998          US    32307766  R1SND1KOPB3EAH  B007A5WCCC      618086229   
4999          US    23079958  R2BDTB2PJ5QFF5  B005H40K7W      648253967   

                                          product_title  star_rating  \
0                  TurboTax

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();